<a href="https://colab.research.google.com/github/Valentino-Cheek/CS-290-LLMs/blob/main/multihead_attention_teardown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
import torch
import torch.nn as nn

In [193]:
# # copited from LLMs-from-scratch/ch03/
# class MultiHeadAttention(nn.Module):
#     def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
#         super().__init__()
#         assert (d_out % num_heads == 0), \
#             "d_out must be divisible by num_heads"

#         self.d_out = d_out
#         self.num_heads = num_heads
#         self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

#         self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
#         self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
#         self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
#         self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
#         self.dropout = nn.Dropout(dropout)
#         self.register_buffer(
#             "mask",
#             torch.triu(torch.ones(context_length, context_length),
#                        diagonal=1)
#         )

#     def forward(self, x):
#         b, num_tokens, d_in = x.shape
#         # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`,
#         # this will result in errors in the mask creation further below.
#         # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
#         # do not exceed `context_length` before reaching this forward method.

#         keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
#         queries = self.W_query(x)
#         values = self.W_value(x)

#         # We implicitly split the matrix by adding a `num_heads` dimension
#         # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
#         keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
#         values = values.view(b, num_tokens, self.num_heads, self.head_dim)
#         queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

#         # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
#         keys = keys.transpose(1, 2)
#         queries = queries.transpose(1, 2)
#         values = values.transpose(1, 2)

#         # Compute scaled dot-product attention (aka self-attention) with a causal mask
#         attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

#         # Original mask truncated to the number of tokens and converted to boolean
#         mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

#         # Use the mask to fill attention scores
#         attn_scores.masked_fill_(mask_bool, -torch.inf)

#         attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
#         attn_weights = self.dropout(attn_weights)

#         # Shape: (b, num_tokens, num_heads, head_dim)
#         context_vec = (attn_weights @ values).transpose(1, 2)

#         # Combine heads, where self.d_out = self.num_heads * self.head_dim
#         context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
#         context_vec = self.out_proj(context_vec) # optional projection

#         return context_vec

# torch.manual_seed(123)

# batch_size, context_length, d_in = batches.shape
# d_out = 2
# mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

# context_vecs = mha(batches)

# print(context_vecs)
# print("context_vecs.shape:", context_vecs.shape)

In [194]:
# Before your code review on Friday, be sure that your GitHub repo includes everything we've done in chapters 2 and 3. In particular,
# your repo should include a notebook in which you implement multihead attention (as discussed in class and in the book) and do the following to explore how it works:

#     Create a batch of inputs. This determines d_in and context_length.
#     Decide on an output dimension, d_out.
#     Pick a number of attention heads, num_heads, such that d_out is divisible by num_heads.
#     In a sequence of cells, go step-by-step through the same calculation of queries and attention scores that the
#     MultiHeadAttention class does. At each step, print the tensors and their shapes to see why reshaping is necessary.


In [195]:
#create a random vector matrix with dimensions 6 x 10
# i.e. four vectors of length 10
d_in = 5
context_length = 3

inputs = torch.nn.Embedding( context_length, d_in )
inputs = inputs.weight.data
inputs

tensor([[ 1.1025, -0.4555, -0.7091,  0.5402, -0.2880],
        [ 3.3063,  1.0562, -0.5891, -0.5763,  0.0959],
        [-1.0870,  0.2619,  0.4741, -1.2685,  1.2751]])

In [196]:
# create batches of this same input
# results in 3 matrices which have 6 vectors of length 10 each
batches = torch.stack((inputs, inputs, inputs), dim=0)
print(batches)
print("batches shape: ", batches.shape)
# torch.stack?

tensor([[[ 1.1025, -0.4555, -0.7091,  0.5402, -0.2880],
         [ 3.3063,  1.0562, -0.5891, -0.5763,  0.0959],
         [-1.0870,  0.2619,  0.4741, -1.2685,  1.2751]],

        [[ 1.1025, -0.4555, -0.7091,  0.5402, -0.2880],
         [ 3.3063,  1.0562, -0.5891, -0.5763,  0.0959],
         [-1.0870,  0.2619,  0.4741, -1.2685,  1.2751]],

        [[ 1.1025, -0.4555, -0.7091,  0.5402, -0.2880],
         [ 3.3063,  1.0562, -0.5891, -0.5763,  0.0959],
         [-1.0870,  0.2619,  0.4741, -1.2685,  1.2751]]])
batches shape:  torch.Size([3, 3, 5])


In [197]:
# define an output dimension and pick a number of attention heads
# d_out msut be divisible by num_heads so that the later view call runs without error
# if it was no the view call would fail because the new shape attempting to be created would have a different number of elements
d_out = 8
num_heads = 2
# head dimensions = the length of each head once view is called
head_dim = d_out // num_heads
head_dim

4

In [198]:
# create the initial query and key weight matrices
W_query = nn.Linear(d_in, d_out, bias=False)
W_keys = nn.Linear(d_in, d_out, bias=False)

In [199]:
# create the queries and keys vectors
keys = W_keys(batches)
queries = W_query(batches)
print("keys: ", keys, "\n keys shape: ",keys.shape)
print("\n queries :", queries, "\n queries shape: ", queries.shape)


keys:  tensor([[[ 0.4326, -0.5948, -0.3360, -0.1327, -0.9645,  0.5664,  0.2623,
           0.7277],
         [ 0.7467, -0.6135, -1.1176, -1.2574, -0.6431,  0.9303,  1.5005,
           0.8179],
         [-0.8376,  0.5426,  1.2401,  0.2090,  1.2661, -0.9023, -0.1516,
          -1.1461]],

        [[ 0.4326, -0.5948, -0.3360, -0.1327, -0.9645,  0.5664,  0.2623,
           0.7277],
         [ 0.7467, -0.6135, -1.1176, -1.2574, -0.6431,  0.9303,  1.5005,
           0.8179],
         [-0.8376,  0.5426,  1.2401,  0.2090,  1.2661, -0.9023, -0.1516,
          -1.1461]],

        [[ 0.4326, -0.5948, -0.3360, -0.1327, -0.9645,  0.5664,  0.2623,
           0.7277],
         [ 0.7467, -0.6135, -1.1176, -1.2574, -0.6431,  0.9303,  1.5005,
           0.8179],
         [-0.8376,  0.5426,  1.2401,  0.2090,  1.2661, -0.9023, -0.1516,
          -1.1461]]], grad_fn=<UnsafeViewBackward0>) 
 keys shape:  torch.Size([3, 3, 8])

 queries : tensor([[[-0.1259,  0.0497, -0.3309,  0.0205, -0.2086, -0.0671,  0.230

In [200]:
# keys.view?

In [201]:
# b = batch size / number of batches - 3
# num tokens = number of vectors per batch - 3
# d_in = the lenght of each vector / input dimensionality
b, num_tokens, d_in = batches.shape
print(b, num_tokens, d_in)

# use view to tranform the keys and queries vectors
# this takes the last dimension, d_out and splits it into two dimensions, depending on the num_heads wanted
keys = keys.view(b, num_tokens, num_heads, head_dim)
queries = queries.view(b, num_tokens, num_heads, head_dim)

print("keys: ", keys, "\n keys shape: ",keys.shape)
print("\n queries: ", queries, "\n queries shape: ", queries.shape)

3 3 5
keys:  tensor([[[[ 0.4326, -0.5948, -0.3360, -0.1327],
          [-0.9645,  0.5664,  0.2623,  0.7277]],

         [[ 0.7467, -0.6135, -1.1176, -1.2574],
          [-0.6431,  0.9303,  1.5005,  0.8179]],

         [[-0.8376,  0.5426,  1.2401,  0.2090],
          [ 1.2661, -0.9023, -0.1516, -1.1461]]],


        [[[ 0.4326, -0.5948, -0.3360, -0.1327],
          [-0.9645,  0.5664,  0.2623,  0.7277]],

         [[ 0.7467, -0.6135, -1.1176, -1.2574],
          [-0.6431,  0.9303,  1.5005,  0.8179]],

         [[-0.8376,  0.5426,  1.2401,  0.2090],
          [ 1.2661, -0.9023, -0.1516, -1.1461]]],


        [[[ 0.4326, -0.5948, -0.3360, -0.1327],
          [-0.9645,  0.5664,  0.2623,  0.7277]],

         [[ 0.7467, -0.6135, -1.1176, -1.2574],
          [-0.6431,  0.9303,  1.5005,  0.8179]],

         [[-0.8376,  0.5426,  1.2401,  0.2090],
          [ 1.2661, -0.9023, -0.1516, -1.1461]]]], grad_fn=<ViewBackward0>) 
 keys shape:  torch.Size([3, 3, 2, 4])

 queries:  tensor([[[[-0.1259,  0.

In [202]:
# Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
# we do this so that when we do the dot product the number of heads is preserved while the number of tokens is in the dot product
# keys and queries need to be multiplied along tokens per head
# this is required because when you make the different heads by calling view num_heads is the 3rd dimension
# you cannot form keys and queries correctly without view therefore this transpose is necessary
# this enables the next step in which the dot product for head in each batch is calculated

# we want each batch to have a number of heads that each have a number(context lengths/num_contexts) with a certain length(head_dim)
# in the original shape each batch has a number of tokens that each have a head with a certain length
keysT = keys.transpose(1, 2)
queriesT = queries.transpose(1, 2)

print("keys: ", keys, "\n keys shape: ",keys.shape)
print("\n queries: ", queries, "\n queries shape: ", queries.shape)

keys:  tensor([[[[ 0.4326, -0.5948, -0.3360, -0.1327],
          [-0.9645,  0.5664,  0.2623,  0.7277]],

         [[ 0.7467, -0.6135, -1.1176, -1.2574],
          [-0.6431,  0.9303,  1.5005,  0.8179]],

         [[-0.8376,  0.5426,  1.2401,  0.2090],
          [ 1.2661, -0.9023, -0.1516, -1.1461]]],


        [[[ 0.4326, -0.5948, -0.3360, -0.1327],
          [-0.9645,  0.5664,  0.2623,  0.7277]],

         [[ 0.7467, -0.6135, -1.1176, -1.2574],
          [-0.6431,  0.9303,  1.5005,  0.8179]],

         [[-0.8376,  0.5426,  1.2401,  0.2090],
          [ 1.2661, -0.9023, -0.1516, -1.1461]]],


        [[[ 0.4326, -0.5948, -0.3360, -0.1327],
          [-0.9645,  0.5664,  0.2623,  0.7277]],

         [[ 0.7467, -0.6135, -1.1176, -1.2574],
          [-0.6431,  0.9303,  1.5005,  0.8179]],

         [[-0.8376,  0.5426,  1.2401,  0.2090],
          [ 1.2661, -0.9023, -0.1516, -1.1461]]]], grad_fn=<ViewBackward0>) 
 keys shape:  torch.Size([3, 3, 2, 4])

 queries:  tensor([[[[-0.1259,  0.0497, 

In [203]:
# Compute scaled dot-product attention (aka self-attention) with a causal mask
# keys. transpose 2,3 (b, num_heads, num_tokens, head_dim) -> (b, num_heads, head_dim, num_tokens)
print("queries shape: ", queriesT.shape, "\nkeys transposed 2,3 shape: ",keysT.transpose(2, 3).shape)

# Dot product for each head, specifically matrix multiplication is only 2D and @ defaults to the last 2 dimensions
# in essence you're doing the dot product for each batches' heads
attn_scores = queriesT @ keysT.transpose(2, 3)

print("\n\n attn_scores: ", attn_scores, "\n\n attn_scores shape: ",attn_scores.shape)


queries shape:  torch.Size([3, 2, 3, 4]) 
keys transposed 2,3 shape:  torch.Size([3, 2, 4, 3])


 attn_scores:  tensor([[[[ 0.0244,  0.2194, -0.2736],
          [-0.1560, -0.5426, -0.0390],
          [ 0.4361,  0.3983, -0.5888]],

         [[ 0.2381,  0.4339, -0.2613],
          [ 0.1851,  0.6994, -0.2565],
          [ 0.4414,  0.1244, -0.8326]]],


        [[[ 0.0244,  0.2194, -0.2736],
          [-0.1560, -0.5426, -0.0390],
          [ 0.4361,  0.3983, -0.5888]],

         [[ 0.2381,  0.4339, -0.2613],
          [ 0.1851,  0.6994, -0.2565],
          [ 0.4414,  0.1244, -0.8326]]],


        [[[ 0.0244,  0.2194, -0.2736],
          [-0.1560, -0.5426, -0.0390],
          [ 0.4361,  0.3983, -0.5888]],

         [[ 0.2381,  0.4339, -0.2613],
          [ 0.1851,  0.6994, -0.2565],
          [ 0.4414,  0.1244, -0.8326]]]], grad_fn=<UnsafeViewBackward0>) 

 attn_scores shape:  torch.Size([3, 2, 3, 3])


In [204]:
# create a mask, where every value north of the diagonal (not including the diagonal) is one
# this stops the model from "cheating" i.e. looking ahead at the words that it shouldn't have access too
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
print(mask)


# Original mask truncated to the number of tokens (ensures the dimensions match) and converted to boolean
mask_bool = mask.bool()[:num_tokens, :num_tokens]
print(mask_bool)

# troubleshooting
print("attn_scores shape:", attn_scores.shape)
print("mask_bool shape:", mask_bool.shape)

# Use the mask to fill attention scores
attn_scores.masked_fill_(mask_bool, -torch.inf)
print(attn_scores)


tensor([[0., 1., 1.],
        [0., 0., 1.],
        [0., 0., 0.]])
tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])
attn_scores shape: torch.Size([3, 2, 3, 3])
mask_bool shape: torch.Size([3, 3])
tensor([[[[ 0.0244,    -inf,    -inf],
          [-0.1560, -0.5426,    -inf],
          [ 0.4361,  0.3983, -0.5888]],

         [[ 0.2381,    -inf,    -inf],
          [ 0.1851,  0.6994,    -inf],
          [ 0.4414,  0.1244, -0.8326]]],


        [[[ 0.0244,    -inf,    -inf],
          [-0.1560, -0.5426,    -inf],
          [ 0.4361,  0.3983, -0.5888]],

         [[ 0.2381,    -inf,    -inf],
          [ 0.1851,  0.6994,    -inf],
          [ 0.4414,  0.1244, -0.8326]]],


        [[[ 0.0244,    -inf,    -inf],
          [-0.1560, -0.5426,    -inf],
          [ 0.4361,  0.3983, -0.5888]],

         [[ 0.2381,    -inf,    -inf],
          [ 0.1851,  0.6994,    -inf],
          [ 0.4414,  0.1244, -0.8326]]]], grad_fn=<MaskedFillBackward0>)
